In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
import json
import requests
import re

In [2]:
es = Elasticsearch("http://localhost:9200")
ic = IndicesClient(es)


In [ ]:
r = requests.get('http://localhost:9200')
if r.status_code == 200:
    # each insurer emits a different format so this will only work for modahealth
    r = requests.get('https://www.modahealth.com/cms-data-index.json')
    urls = json.loads(r.content)
#    provider
    for provider_url in urls['provider_urls']:
        print provider_url
        split_url = re.split('-',provider_url)
        state = (split_url[-1].split('.'))[0].lower()
        r = requests.get(provider_url)
        for item in json.loads(r.content):
            es.index(index=state, doc_type='provider', body=item)
#    formulary
    for formulary_url in urls['formulary_urls']:
        print formulary_url
        split_url = re.split('-',formulary_url)
        state = (split_url[-1].split('.'))[0].lower()
        r = requests.get(formulary_url)
        for item in json.loads(r.content):
            es.index(index=state, doc_type='formulary', body=item)
#    plan
    for plan_url in urls['plan_urls']:
        print plan_url
        split_url = re.split('-',plan_url)
        state = (split_url[-1].split('.'))[0].lower()
        r = requests.get(plan_url)
        for item in json.loads(r.content):
            es.index(index=state, doc_type='plan', body=item)

In [ ]:
-

In [25]:
i = 1
with open ('plans-AK.json') as infile:
    plans = json.load(infile)
    for item in plans:
        es.index(index="ak", doc_type="plans", id=i, body=item)
        i += 1
#    print json.dumps(plans)

In [26]:
es.search(index="ak", body={"query": {"match": {'marketing_name':'Be Equipped'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'4',
    u'_index': u'ak',
    u'_score': 0.9832023,
    u'_source': {u'formulary': [{u'cost_sharing': [{u'coinsurance_opt': None,
         u'coinsurance_rate': 0.0,
         u'copay_amount': 180.0,
         u'copay_opt': u'NO-CHARGE',
         u'pharmacy_type': u'3-MONTH-IN-MAIL'},
        {u'coinsurance_opt': None,
         u'coinsurance_rate': 0.0,
         u'copay_amount': 60.0,
         u'copay_opt': u'BEFORE-DEDUCTIBLE',
         u'pharmacy_type': u'3-MONTH-IN-RETAIL'},
        {u'coinsurance_opt': None,
         u'coinsurance_rate': 0.0,
         u'copay_amount': 60.0,
         u'copay_opt': u'BEFORE-DEDUCTIBLE',
         u'pharmacy_type': u'3-MONTH-OUT-RETAIL'}],
       u'drug_tier': u'SELECT',
       u'mail_order': True},
      {u'cost_sharing': [{u'coinsurance_opt': None,
         u'coinsurance_rate': 0.0,
         u'copay_amount': 0.0,
         u'copay_opt': u'NO-CHARGE',
         u'ph

In [9]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
import os
import json

In [17]:

auth = Oauth1Authenticator(
    consumer_key=os.environ['YELP_CONSUMER_KEY'],
    consumer_secret=os.environ['YELP_CONSUMER_SECRET'],
    token=os.environ['YELP_TOKEN'],
    token_secret=os.environ['YELP_TOKEN_SECRET']
)

client = Client(auth)

In [21]:
params = {
    'term': 'Zenner M.D.',
    'lang': 'en'
}

a=client.search('Virginia Beach', **params)

In [19]:
for item in a.businesses:
    print item.name, item.review_count

Coastal Grill 76
Blue Pearl Veterinary Hospital 21
Tautog's Restaurant 481
Onelife Fitness - Greenbrier 42
Bodega 89
Carrabba's Italian Grill 36
Cheddar's Scratch Kitchen 192
Berrets Seafood Restaurant and Taphouse Grill 199
